In [29]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image

class HierarchicalResNet(nn.Module):
    def __init__(self, num_classes):
        super(HierarchicalResNet, self).__init__()
        self.resnet = resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        self.fc_class = nn.Linear(num_ftrs, num_classes) 
        
    def forward(self, x):
        features = self.resnet(x)
        class_output = self.fc_class(features)
        return class_output

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
])

class HierarchicalDataset(Dataset):
    def __init__(self, data_dirs, transform=None):
        self.data = []
        self.class_map = {}
        class_idx = 0
        for class_dir in data_dirs:
            class_label = os.path.basename(class_dir)
            self.class_map[class_idx] = class_label
            for sub_dir in os.listdir(class_dir):
                sub_dir_path = os.path.join(class_dir, sub_dir)
                if os.path.isdir(sub_dir_path):
                    for img_file in os.listdir(sub_dir_path):
                        img_path = os.path.join(sub_dir_path, img_file)
                        _, ext = os.path.splitext(img_path)
                        if ext.lower() in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']: 
                            img = Image.open(img_path)
                            if img.mode == 'RGB': 
                                self.data.append((img_path, class_idx))
            class_idx += 1
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_idx = self.data[idx]
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
        return img, class_idx

train_dirs = [
    '../Image Data/Dance/train',
    '../Image Data/Monuments/train',
    '../Image Data/Paintings/training'
]

train_dataset = HierarchicalDataset(train_dirs, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
num_classes = 3  
model = HierarchicalResNet(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2
print_every = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    minibatch_counter = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item() * inputs.size(0)
        minibatch_counter += 1
        if minibatch_counter % print_every == 0:
            minibatch_loss = running_loss / total
            minibatch_accuracy = correct / total
            print(f'Epoch [{epoch+1}/{num_epochs}], Minibatch [{minibatch_counter}/{len(train_loader)}], Loss: {minibatch_loss:.4f}, Accuracy: {minibatch_accuracy:.4f}')
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
torch.save(model.state_dict(), 'HierarchicalClassificationModel.pth')

Epoch [1/2], Minibatch [10/90], Loss: 0.8829, Accuracy: 0.6062
Epoch [1/2], Minibatch [20/90], Loss: 0.7106, Accuracy: 0.7516
Epoch [1/2], Minibatch [30/90], Loss: 0.5896, Accuracy: 0.7974
Epoch [1/2], Minibatch [40/90], Loss: 0.5066, Accuracy: 0.8313
Epoch [1/2], Minibatch [50/90], Loss: 0.4479, Accuracy: 0.8516
Epoch [1/2], Minibatch [60/90], Loss: 0.3986, Accuracy: 0.8714
Epoch [1/2], Minibatch [70/90], Loss: 0.3676, Accuracy: 0.8824
Epoch [1/2], Minibatch [80/90], Loss: 0.3372, Accuracy: 0.8943
Epoch [1/2], Minibatch [90/90], Loss: 0.3139, Accuracy: 0.9027
Epoch [1/2], Loss: 0.3139, Accuracy: 0.9027
Epoch [2/2], Minibatch [10/90], Loss: 0.1102, Accuracy: 0.9766
Epoch [2/2], Minibatch [20/90], Loss: 0.1083, Accuracy: 0.9773
Epoch [2/2], Minibatch [30/90], Loss: 0.0961, Accuracy: 0.9818
Epoch [2/2], Minibatch [40/90], Loss: 0.0933, Accuracy: 0.9824
Epoch [2/2], Minibatch [50/90], Loss: 0.0918, Accuracy: 0.9822
Epoch [2/2], Minibatch [60/90], Loss: 0.0903, Accuracy: 0.9823
Epoch [2/2]

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.resnet = resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
])

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(root=data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dir = '../Image Data/Paintings/training'
val_dir = '../Image Data/Paintings/testing'
train_dataset = CustomDataset(train_dir, transform=transform)
val_dataset = CustomDataset(val_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
num_classes = 8
model = CustomResNet(num_classes)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
print_every = 7 

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    minibatch_counter = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        minibatch_counter += 1
        if minibatch_counter % print_every == 0:
            minibatch_loss = running_loss / total
            minibatch_accuracy = correct / total
            print(f'Epoch [{epoch+1}/{num_epochs}], Minibatch [{minibatch_counter}/{len(train_loader)}], Loss: {minibatch_loss:.4f}, Accuracy: {minibatch_accuracy:.4f}')

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')


print('Fine-tuning finished.')
torch.save(model.state_dict(), 'Painting.pth')

Epoch [1/10], Minibatch [7/28], Loss: 2.1721, Accuracy: 0.1228
Epoch [1/10], Minibatch [14/28], Loss: 2.0139, Accuracy: 0.2210


KeyboardInterrupt: 

In [30]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
])

num_classes = 3  
model = HierarchicalResNet(num_classes)  
model.load_state_dict(torch.load('HierarchicalClassificationModel.pth'))
model.eval()

train_dataset = ImageFolder(root='../Image Data/Paintings/training', transform=transform)
class_names = train_dataset.classes

def predict_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    predicted_idx = predicted.item()
    if 0 <= predicted_idx < len(class_names):  
        predicted_class = class_names[predicted_idx]
    else:
        predicted_class = "Image does not belong to any known class"
    return predicted_class

image_paths = ['../Image Data/Monuments/train/Ajanta Caves/(1).jpg', '../Image Data/Paintings/training/Madhubani/p1.jpg']  
for image_path in image_paths:
    if 'Paintings' in image_path:
        predicted_class = predict_image(image_path)
        print(f'Image: {image_path}, Predicted Class: {predicted_class}')
    else:
        print(f'Image: {image_path}, Image belongs to a different category.')


Image: ../Image Data/Monuments/train/Ajanta Caves/(1).jpg, Image belongs to a different category.
Image: ../Image Data/Paintings/training/Madhubani/p1.jpg, Predicted Class: Madhubani
